# Analyse des données slavevoyages

URL de la requête : https://www.slavevoyages.org/voyage/database
        
Filtres : 
- filtre "Place where voyage began" : La Rochelle, Rochefort, Bordeaux, Nantes
- Export en csv

J'ai ensuite exporté un csv avec les colonnes suivantes : Voyage itinerary imputed port where began (ptdepimp) place,"Voyage dates that began (depb,a,c)"

Je l'ai simplifié au niveau des colonnes.

Puis chargement du fichier :

In [ ]:
import csv
import re


voyages = []
with open('slavevoyages_exports_filtered.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        year = re.search("([\d]{4})", row["date_departure"] if "date_departure" in row else '')
        year = year.group(0) if year else None
        row["port"] = "La Rochelle et Rochefort" if (row["port"] == 'La Rochelle' or row['port'] == 'Rochefort') else row['port']
        voyages.append({**row, "year": year, "has_year": "année renseignée" if year is not None else "pas d'année"})

# 1. Récupération et vérification des données de slavevoyages

In [ ]:
from vega import VegaLite
import pandas as pds

VegaLite({
    "title": "Complétude",
    "mark": "bar",
    "encoding": {
        "column": {
            "type": "nominal", 
            "field": "port",
            "title": "port"
        },
        "y": {
            "field": "has_year",
            "type": "nominal",
            "title": "année de départ renseignée"
        },
        "x": {
            "field": "has_year",
            "type": "quantitative",
            "aggregate": "count",
            "title": "nombre de voyages"
        },
    }
}, pds.DataFrame(voyages))

In [ ]:
VegaLite({
    "title": "Évolution des départs de navires négriers",
    "mark": "line",
    "width": 800,
    "encoding": {
        "row": {
            "type": "nominal", 
            "field": "port",
            "title": "port"
        },
        "y": {
            "field": "year",
            "type": "quantitative",
            "aggregate": "count",
            "title": "nombre de voyages"
        },
        "x": {
            "field": "year",
            "type": "temporal"
        },
    }
}, pds.DataFrame([v for v in voyages if v["year"] is not None]))

# 2. Évolution de la part des navires négriers à partir des données navigo

On récupère tous les pointcalls Out pour les ports de La Rochelle, Nantes, Bordeaux, de 1700 à 1800

In [ ]:
from poitousprint import Portic
import re

client = Portic()

pointcalls = client.get_pointcalls(
    start_year=1700, 
    end_year=1800,
    homeport_toponyme_fr=["La Rochelle", "Nantes", "Bordeaux", "Rochefort"],
    # pointcall_action='Out',
    params=['homeport_toponyme_fr', 'year', 'pointcall_action', 'outdate_fixed', 'pointcall_out_date']
)

In [ ]:
def parse_year(p):
    # 1787=03=23
    year = re.search("([\d]{4})", p['pointcall_out_date'])[0] if p['pointcall_out_date'] is not None else None
    port = p["homeport_toponyme_fr"]
    if port == "La Rochelle" or port == "Rochefort":
        port = "La Rochelle et Rochefort"
    return {**p.copy(), "year": int(year), "homeport_toponyme_fr": port}

pointcalls = [parse_year(p) for p in pointcalls if p['pointcall_action'] == 'Out']

print(str(len(pointcalls)) + ' pointcalls trouvés')

In [ ]:
VegaLite({
    "title": "Distribution des pointcalls OUT pour les ports de Nantes, Bordeaux, et La Rochelle",
    "mark": "bar",
    "encoding": {
        "y": {
            "field": "year",
            "type": "quantitative",
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "x": {
            "field": "year",
            "type": "ordinal"
        },
        "color": {
            "field": "homeport_toponyme_fr",
            "type": "nominal"
        },
    }
}, pds.DataFrame(pointcalls))

In [ ]:
VegaLite({
    "title": "Distribution des pointcalls OUT pour les ports de Nantes, Bordeaux, et La Rochelle",
    "mark": "bar",
    "encoding": {
        "y": {
            "field": "year",
            "type": "quantitative",
            "aggregate": "count",
            "title": "nombre de pointcalls",
            "stack": "normalize"
        },
        "x": {
            "field": "year",
            "type": "ordinal"
        },
        "color": {
            "field": "homeport_toponyme_fr",
            "type": "nominal"
        },
    }
}, pds.DataFrame(pointcalls))

In [ ]:
# Building slaves data
slaves_dict = {}
for v in [v for v in voyages if v["year"] is not None]:
    if v["year"] not in slaves_dict:
        slaves_dict[v["year"]] = {}
    if v["port"] not in slaves_dict[v["year"]]:
        slaves_dict[v["year"]][v["port"]] = {
            "slave_travels": 1
        }
    else :
        slaves_dict[v["year"]][v["port"]]["slave_travels"] += 1

for p in pointcalls:
    """
    if p["year"] not in slaves_dict:
        slaves_dict[p["year"]] = {}
    if p["homeport_toponyme_fr"] not in slaves_dict[p["year"]]:
        slaves_dict[p["year"]][p["homeport_toponyme_fr"]] = {
            "slave_travels": 0
        }
    """
    if "nb_pointcalls_out" not in slaves_dict[str(p["year"])][p["homeport_toponyme_fr"]]:
        slaves_dict[str(p["year"])][p["homeport_toponyme_fr"]]["nb_pointcalls_out"] = 1
    else:
        slaves_dict[str(p["year"])][p["homeport_toponyme_fr"]]["nb_pointcalls_out"] += 1
    slaves_dict[str(p["year"])][p["homeport_toponyme_fr"]]["slave_travels_ratio"] = slaves_dict[str(p["year"])][p["homeport_toponyme_fr"]]["slave_travels"] / slaves_dict[str(p["year"])][p["homeport_toponyme_fr"]]["nb_pointcalls_out"]

by_year = [{"year": year, "ports": ports} for (year, ports) in slaves_dict.items()]
flat_travels = []
for year in by_year:
    ports = [{"year": year["year"], "port": port, **data} for (port, data) in year["ports"].items()]
    flat_travels += ports
    
flat_travels = [t for t in flat_travels if "nb_pointcalls_out" in t]

In [ ]:
VegaLite({
    "title": "Proportion des expéditions négrières parmi les pointcalls OUT des trois ports",
    "mark": "bar",
    "encoding": {
        "y": {
            "field": "slave_travels_ratio",
            "type": "quantitative",
            "title": "proportion de voyages négriers",
            "scale": {
                "domain": [0, 1]
            }
        },
        "x": {
            "field": "port",
            "type": "nominal"
        },
        "color": {
            "field": "port",
            "type": "nominal"
        },
        "column": {
            "field": "year",
            "type": "nominal"
        },
    }
}, pds.DataFrame(flat_travels))


Conclusion : pas très concluant ?